# Non-Parametric Trend Analysis

Based on https://developers.google.com/earth-engine/tutorials/community/nonparametric-trends

In [1]:
import ee
import json
import geemap
from pathlib import Path
from ee._helpers import ServiceAccountCredentials

In [2]:
# Connect to GEE
service_credentials_file=Path("../secrets/ee-observatorionieves-288939dbc1cf.json")

with open(service_credentials_file, "r") as f:
    credential_data = json.load(f)

#print( service_account_data)
credentials = ServiceAccountCredentials(
    email=credential_data['client_email'],
    key_data=json.dumps(credential_data),
)
ee.Initialize(credentials=credentials, project=credential_data["project_id"])

Trend analysis is finding places where something of interest is increasing or decreasing and by how much. More specifically, this tutorial demonstrates detecting monotonic trends in imagery using the non-parametric Mann-Kendall test for the presence of an increasing or decreasing trend and Sen's slope to quantify the magnitude of the trend (if one exists). The tutorial also shows estimating the variance of the Mann-Kendall test statistic, a Z-statistic for the test of presence of any trend, and a P-value of the statistic (assuming a normal distribution).

Mann-Kendal statistic (S) is the sum of the signs of the differences between all pairs of data points (+1, 0, -1).

In [3]:
ee_mod13_ic = ee.imagecollection.ImageCollection('MODIS/006/MOD13A1');
ee_coll_ic = ee.imagecollection.ImageCollection(ee_mod13_ic.select('EVI').filter(ee.filter.Filter.calendarRange(8, 9, 'month')))

map = geemap.Map()
map.setCenter(-121.6, 37.3, 10)
map.addLayer(ee_object=ee_coll_ic, vis_params={'min':-500, 'max': 5000, 'palette': ['white', 'green']}, name='coll')

c:\Users\erick\Documents\Projects\teleamb\observatorio_ipa\.venv\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD13A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13A1

  warnings.warn(warning, category=DeprecationWarning)


In [4]:
ee_coll_ic.size().getInfo()

92

## Join the time series to itself

The non-parametric stats of interest are computed by examining every possible ordered pair of unique values in the time series. If there are n time points in the series, we need to examine the n(n-1)/2 pairs (i, j), i<j, where i and j are arbitrary time indices. (Here we use (i, j) to signify the pair of EVI images indexed by i and j). To do that, join the collection to itself with a temporal filter. The temporal filter will pass all the chronologically later images. In the joined collection, each image will store all the images that come after it in an after property.

In [5]:
ee_afterFilter = ee.filter.Filter.lessThan(
  leftField = 'system:time_start',
  rightField = 'system:time_start')

ee_joined_ic = ee.imagecollection.ImageCollection(ee.join.Join.saveAll('after').apply(
  primary= ee_coll_ic,
  secondary= ee_coll_ic,
  condition= ee_afterFilter
))

## Mann-Kendall trend test

The Mann-Kendall trend is defined as the sum of the signs of all the pairs. The sign is 1 if EVI at time j is more than EVI at time i, -1 if the opposite is true and zero otherwise (if they're equal). Compute this by iterating over the collection and computing sign(i, j) for each image i in the collection and each image j in the after images.

In [6]:
# The unmask is to prevent accumulation of masked pixels that
# result from the undefined case of when either current or image
# is masked.  It won't affect the sum, since it's unmasked to zero.


def _ee_sign(ee_i_img: ee.image.Image, ee_j_img: ee.image.Image) -> ee.image.Image:
    """Compute the sign of the difference between two images."""
    return (ee.image.Image(ee_j_img).neq(ee_i_img)  # Zero case
        .multiply(ee.image.Image(ee_j_img).subtract(ee_i_img).clamp(-1, 1))
        .int())
    


def _ee_calc_ts_signs(ee_current_img: ee.image.Image):
    ee_afterCollection_ic = ee.imagecollection.ImageCollection.fromImages(ee_current_img.get('after'));
    ee_afterCollection_ic = ee_afterCollection_ic.map(
        lambda ee_after_img: ee.image.Image(_ee_sign(ee_current_img, ee_after_img)).unmask(0)
    )
    
    return ee_afterCollection_ic


ee_signs_img = ee.imagecollection.ImageCollection(
  ee_joined_ic.map(
      lambda ee_current_img:  _ee_calc_ts_signs(ee_current_img)
    ).flatten())

ee_kendall_img =  ee_signs_img.reduce('sum', 2) #type: ignore



In [9]:
palette = ['red', 'white', 'green']
# Set min and max stretch visualization parameters as necessary.
map.addLayer(
    ee_kendall_img, 
    {'palette': palette, 'min': -2800, 'max': 2800}, 
    'kendall')

## Sen's slope

Sen's slope is computed in a similar way to the Mann-Kendall statistic. However, instead of adding all the signs of the pairs, the slope is computed for all the pairs. Sen's slope is the median slope from all those pairs. In the following, slope is computed over days to avoid numerically tiny slopes (which might result from using epoch time instead).

In [10]:
def  _ee_calc_slope (ee_i_img, ee_j_img):
  return (ee.image.Image(ee_j_img).subtract(ee_i_img)
      .divide(ee.image.Image(ee_j_img).date().difference(ee.image.Image(ee_i_img).date(), 'days')) #! Verify what unit is being used in code
      .rename('slope')
      .float())


def _ee_calc_ts_slopes(ee_current_img):
    ee_afterCollection_ic = ee.imagecollection.ImageCollection.fromImages(ee_current_img.get('after'));
    ee_afterCollection_ic = ee_afterCollection_ic.map(
        lambda ee_after_img: ee.image.Image(_ee_calc_slope(ee_current_img, ee_after_img))
    )
    
    return ee_afterCollection_ic


ee_slopes_ic = ee.imagecollection.ImageCollection(
   ee_joined_ic.map(lambda ee_image: _ee_calc_ts_slopes(ee_image)).flatten()
)
 
ee_sensSlope_img = ee_slopes_ic.reduce(ee.reducer.Reducer.median(), 2)


In [11]:

map.addLayer(
    ee_sensSlope_img, 
    {"palette": palette, "min": -1, "max": 1}, 
    'sensSlope')

## Variance of the Mann-Kendall statistic

Computing the variance of the Mann-Kendall statistic is complicated by the possible presence of ties in the data (i.e. sign(i, j) equals zero). Counting those ties can get a little dicey, requiring an array-based forward differencing. Note that you can comment .subtract(groupFactorSum) in the computation of kendallVariance if you don't care about ties and want to disregard that correction

In [13]:
# The formula for the Variance of Mann-Kendall requires identifying groups of tied values in a timeseries.
# The below code does this by first identifying the pixels and values that have ties, then determines the group 
# siz (Number of times the values repeat) using arrays. Then computes the factors for the groups that are used 
# in the variance formula, and finally computes the variance.

# An simple example of groups for a single pixel in a timeseries:
# [0.35,0.40,0.40,0.42,0.42,0.42,0.44]
# - This has two tie groups
# - One group of two tied values at 0.40 -> t=2
# - One group of three tied values at 0.42 -> t=3


# Keep pixel values where the value matched in more than 1 other image in the collection. Set all else to zero.
# These are pixels that should have at least one group of "ties"

def _matches(ee_i_img: ee.image.Image, ee_j_img: ee.image.Image) -> ee.image.Image:
    return ee_i_img.eq(ee_j_img)  # i and j are images.

def _groups(ee_i_img: ee.image.Image, ee_icollection:ee.imagecollection.ImageCollection) -> ee.image.Image:
    """Keep values of pixels that had more than one match in other images. 
    
    >1 because at least one match is the image itself.

    """

    ee_matches_img = ee_icollection.map(lambda ee_j_img: _matches(ee_i_img, ee_j_img)).sum()
    return ee_i_img.multiply(ee_matches_img.gt(1))  


# Compute tie group sizes in a sequence.  The first group is discarded.
def _group_sizes(ee_array_img: ee.image.Image) -> ee.image.Image:
    ee_length_img = ee_array_img.arrayLength(0)
    # Array of indices.  These are 1-indexed.
    ee_indices_img = (
        ee.image.Image([1])
        .arrayRepeat(0, ee_length_img)
        .arrayAccum(0, ee.reducer.Reducer.sum())
        .toArray(1)
    )
    ee_sorted_img = ee_array_img.arraySort()
    ee_left_img = ee_sorted_img.arraySlice(0, 1)
    ee_right_img = ee_sorted_img.arraySlice(0, 0, -1)

    # Indices of the end of runs. Always keep the last index, the end of the sequence.
    ee_mask_img = ee_left_img.neq(ee_right_img).arrayCat(
        ee.image.Image(ee.ee_array.Array([[1]])), 0
    )
    ee_runIndices_img = ee_indices_img.arrayMask(ee_mask_img)

    # Subtract the indices to get run lengths.
    ee_groupSizes_img = ee_runIndices_img.arraySlice(0, 1).subtract(
        ee_runIndices_img.arraySlice(0, 0, -1)
    )
    return ee_groupSizes_img


# See equation 2.6 in Sen (1968).
def _factors(ee_image: ee.image.Image) -> ee.image.Image:
    return ee_image.expression("b() * (b() - 1) * (b() * 2 + 5)")


ee_groups_ic = ee.imagecollection.ImageCollection(ee_coll_ic.map(lambda ee_i_img: _groups(ee_i_img, ee_coll_ic)))
ee_groupSizes_img = _group_sizes(ee_groups_ic.toArray())
ee_groupFactors_img = _factors(ee_groupSizes_img)
ee_groupFactorSum_img = ee_groupFactors_img.arrayReduce('sum', [0]).arrayGet([0, 0])

ee_count_img = ee_joined_ic.count()

ee_kendallVariance_img = _factors(ee_count_img).subtract(ee_groupFactorSum_img).divide(18).float()

In [ ]:

map.addLayer(ee_kendallVariance_img, {"min": 1700, "max": 85000}, 'kendallVariance');

## Significance testing
The Mann-Kendall statistic is asymptotically normal for suitably large samples. Assume our samples are suitably large and uncorrelated. Under these assumptions, the true mean of the Mann-Kendall statistic is zero and the variance is as computed above. To compute a standard normal statistic (z), divide the statistic by its standard deviation. The P-value of the z-statistic (probability of observing such an extreme value) is 1 - P(|z| < Z). For a two-sided test of whether any trend exists (positive or negative) at the 95% confidence level, compare the P-value to 0.975. (Alternatively, compare the z-statistic to Z*, where Z* is the inverse distribution function of 0.975).

The standard normal distribution function can be computed in Earth Engine from the error function, erf(). Both the distribution function and its inverse are shown below for reference. Here we use the distribution function to get 1 - P(|z| < Z) and compare it to 0.975.

In [16]:
# Compute Z-statistics.
ee_zero_img = ee_kendall_img.multiply(ee_kendall_img.eq(0))
ee_pos_img = ee_kendall_img.multiply(ee_kendall_img.gt(0)).subtract(1)
ee_neg_img = ee_kendall_img.multiply(ee_kendall_img.lt(0)).add(1)

ee_z_img = ee_zero_img.add(ee_pos_img.divide(ee_kendallVariance_img.sqrt())).add(ee_neg_img.divide(ee_kendallVariance_img.sqrt()))


# https://en.wikipedia.org/wiki/Error_function#Cumulative_distribution_function
def _eeCdf(ee_z_img) -> ee.image.Image:
  return ee.image.Image(0.5).multiply(
    ee.image.Image(1).add(
      ee.image.Image(ee_z_img).divide(ee.image.Image(2).sqrt()).erf()))


# function invCdf(p) {
#   return ee.Image(2).sqrt()
#       .multiply(ee.Image(p).multiply(2).subtract(1).erfInv());
# }

# Compute P-values.
ee_p_img = ee.image.Image(1).subtract(_eeCdf(ee_z_img.abs()))

# pixels with significant trends
# Pixels that can have the null hypothesis (there is no trend) rejected.
# Specifically, if the true trend is zero, there would be less than 5%
# chance of randomly obtaining the observed result (that there is a trend).
ee_significant_trends_img = ee_p_img.lte(0.025)

In [17]:


map.addLayer(ee_z_img, {"min": -2, "max": 2}, 'z')
map.addLayer(ee_p_img, {"min": 0, "max": 1}, 'p')
map.addLayer(ee_significant_trends_img, {"min": 0, "max": 1}, 'significant trends')

In [ ]:
map